In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler, PolynomialFeatures

from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split, RandomizedSearchCV



In [2]:
data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
X = data.drop('Survived', axis = 1)
y = data.Survived

In [4]:
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [5]:
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

In [6]:
# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

In [7]:
# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [8]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [30]:
def ajuste(n, X_train = X_train, X_valid = X_valid, y_train = y_train, y_valid = y_valid):
    model = RandomForestClassifier(min_samples_leaf = n, criterion = 'entropy', n_estimators=300, random_state=0)
    my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])
    my_pipeline.fit(X_train, y_train)
    preds = my_pipeline.predict(X_valid)
    score = mean_absolute_error(y_valid, preds)
    return score

In [35]:
for n in None, None:
    print(f' min_samples_leaf = {n} o MAE fica = {ajuste(n)}')

TypeError: '<' not supported between instances of 'float' and 'NoneType'

In [ ]:
# Obs: NÃO DEVO ALTERAR:
max_leaf_nodes
n_jobs
min_samples_leaf
min_samples_split
max_depth


In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=0)

In [ ]:
# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

In [ ]:

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)
# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)
# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

Prevendo o TEST

In [ ]:
#retirar 3 colunas do test:
test2 = test[my_cols].copy()

preds_test = my_pipeline.predict(test2)

Salvar para submeter

In [ ]:
# Cria e salva arquivo para submissão
salvar = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': preds_test})
salvar.to_csv('submission_4.csv', index=False)
print("Your submission was successfully saved!")

In [ ]:
salvar.head(10)